In [1]:
#User inputs for date. Hopefully this will be the only manual component
month = 1
day = 1
year = 2023

In [2]:
#In an attempt to streamline this code, I will combine preexisting scripts into one notebook
#import the necessary python packages
import sys

selenium_path = r'C:\Users\Kimbe\anaconda3\Lib\site-packages\selenium'
PYSOLR_PATH = r'c:\users\kimbe\appdata\local\programs\python\python312\lib\site-packages'

if not PYSOLR_PATH in sys.path:
    sys.path.append(PYSOLR_PATH)

if not selenium_path in sys.path:
    sys.path.append(selenium_path)

import pandas
import numpy
import requests
import bs4
import re
import undetected_chromedriver as uc
import selenium
import os
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time
import datetime
import re

In [16]:
def get_case_list(url):
    #Declare a list to store the results
    case_data_text = []
    
    #Launch the headless browser
    driver = uc.Chrome()

    #Navigate to the CaseNet URL
    #driver.get(url)
    try:
        driver.get(url)
        driver.maximize_window()
        driver.execute_script("document.body.style.zoom='60%'")
        
    except Exception as e:
        print(e.message)

    #Give the page ample time to load
    time.sleep(10)
    #wait = WebDriverWait(driver, 10)

    #Scroll to ensure the "Next" button is visible
    element = driver.find_element(By.ID, "searchResult_next")
    actions = ActionChains(driver)
    actions.move_to_element(element).perform()
    
    #Wait for the page to load
    time.sleep(2) 
    i = 0
    #Loop through all pages of the table. Find all table rows, and extract all table data, saving the data in a list
    while True:
        try:
            time.sleep(5)
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "searchResult_next"))).click()
            html_content = driver.page_source
            soup = bs4.BeautifulSoup(html_content, 'html.parser')

            #Get all table rows on the page
            trs = soup.find_all(attrs={"role": "row"}) 

            for tr in trs:
                tds = tr.find_all('td') # output data by referring to the list items by position
                case_data_text.append(tds)
                #print(f'Next------------------------------------') # for clarity

        except TimeoutException:
            break
        else:
            print("error " + str(i))
            if i >= 500:
                driver.close()
                return case_data_text
            else:
                 i = i + 1   
            
    driver.close()
    return case_data_text

In [17]:
#Create a date variable for the query, and create an empty array to store webscraper results
start_date = str(month) + '%2F' + str(day) + '%2F' + str(year)

#URL to query cases in St. Louis County (aka the 21st Circuit Court)
casenet_url = 'https://www.courts.mo.gov/cnet/searchResult.do?countyCode=SLC&newSearch=Y&courtCode=CT21&startDate=' + start_date + '&caseStatus=A&caseType=All&locationCode='

In [19]:
#Query all cases occurring in the seven day period starting with the user-defined date.
cases_text = get_case_list(casenet_url)
#I set th eerror max to 1200 when it actually needs to be 500-ish

error 0
error 1
error 2
error 3
error 4
error 5
error 6
error 7
error 8
error 9
error 10
error 11
error 12
error 13
error 14
error 15
error 16
error 17
error 18
error 19
error 20
error 21
error 22
error 23
error 24
error 25
error 26
error 27
error 28
error 29
error 30
error 31
error 32
error 33
error 34
error 35
error 36
error 37
error 38
error 39
error 40
error 41
error 42
error 43
error 44
error 45
error 46
error 47
error 48
error 49
error 50
error 51
error 52
error 53
error 54
error 55
error 56
error 57
error 58
error 59
error 60
error 61
error 62
error 63
error 64
error 65
error 66
error 67
error 68
error 69
error 70
error 71
error 72
error 73
error 74
error 75
error 76
error 77
error 78
error 79
error 80
error 81
error 82
error 83
error 84
error 85
error 86
error 87
error 88
error 89
error 90
error 91
error 92
error 93
error 94
error 95
error 96
error 97
error 98
error 99
error 100
error 101
error 102
error 103
error 104
error 105
error 106
error 107
error 108
error 109
error 110


In [21]:
#Parse the case descriptions 
cases_text
cases_df = pandas.DataFrame({'CASES_HTML':cases_text})
cases_df.to_csv(r"data\case_data\cases_"+start_date+".csv", index = False)

In [ ]:
#Navigate to each case's webpage and scrape any relevant details